In [2]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os
from repeater import retry
from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [9]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cx84858b30b7df25f70063c27c717243fa6c3d7580"
NEWTON_ADDRESS = "hx9393939393939393939393939393939393939399"
DEPLOY_PARAMS = {}

In [4]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))

# Returns block information by block height
block = icon_service.get_block(1000)


In [5]:
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
prajwol = KeyWallet.load('config/yeouido/keystores/testPrajwol', 'bgsa199@#')

print(operator.get_address())
print(icon_service.get_balance(operator.get_address()))
print(icon_service.get_balance(prajwol.get_address()))

hx8b94a3792f336b71937709dae5487166c180c87a
12156515113990000000000
100000000000000000000


In [6]:
DEPLOY_PARAMS =  {
            "_tokenName": "TestToken",
            "_symbolName": "TK",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)


In [8]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x781b2b45c57b3fb02ee27ed763b159f2ef0ac22da4064cfb9c811c157d5bd944',
 'blockHeight': 3713668,
 'blockHash': '0x0f757a7bbec637c8ad337a97280eba7ca35f617ef09dda727aed3d2929d4d02d',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cx84858b30b7df25f70063c27c717243fa6c3d7580',
 'stepUsed': 1222393400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1222393400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [10]:
UPDATE_PARAMS =  {
            "_tokenName": "TestToken1",
            "_symbolName": "TK1",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(UPDATE_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)


In [11]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x9fe3f2ae3bb5050779ac0f9867b6b0c3f1169b4180443e8028b27e2711c1c4c3',
 'blockHeight': 3713677,
 'blockHash': '0xca80549407cbfa5d518ba26146c5f197752857339b7d84f250c430a91ae819b2',
 'txIndex': 1,
 'to': 'cx84858b30b7df25f70063c27c717243fa6c3d7580',
 'scoreAddress': 'cx84858b30b7df25f70063c27c717243fa6c3d7580',
 'stepUsed': 1822386760,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1822386760,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [12]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

TestToken1
TK1
0x12
0x3635c9adc5dea00000


In [13]:
transfer_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method('transfer')\
    .params({'_to':NEWTON_ADDRESS, '_value':1})\
    .build()

estimate_step = icon_service.estimate_step(transfer_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transfer_transaction, operator, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)



In [17]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0xbc8f11e4bf56a8aa3a1a4e44c00888b511d57008c835dfe912e84aa1a4bdf769',
 'blockHeight': 3713692,
 'blockHash': '0x29c96a754c81712f37b790b697ee20d97813f05c1622016d2ec6617f8b227ca5',
 'txIndex': 1,
 'to': 'cx84858b30b7df25f70063c27c717243fa6c3d7580',
 'stepUsed': 153840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 153840,
 'eventLogs': [{'scoreAddress': 'cx84858b30b7df25f70063c27c717243fa6c3d7580',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx8b94a3792f336b71937709dae5487166c180c87a',
    'hx9393939393939393939393939393939393939399',
    '0x1'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [18]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': NEWTON_ADDRESS})\
                    .build()
print(icon_service.call(call))

0x1


In [ ]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': operator.get_address()})\
                    .build()
print(icon_service.call(call))

In [ ]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': prajwol.get_address()})\
                    .build()
print(icon_service.call(call))

In [33]:
params={
    "spender": NEWTON_ADDRESS,
    "owner": operator.get_address(),
}
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('_allowance')\
                    .params(params)\
                    .build()
print(icon_service.call(call))

0x1


In [30]:
params={
    "spender": NEWTON_ADDRESS,
    "value": 1,
}

call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("decreaseAllowance")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0xea8348b0a574ed15870b76efc0e1e486c2f09aa572db9190bcee836640535d3d'

In [32]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0xea8348b0a574ed15870b76efc0e1e486c2f09aa572db9190bcee836640535d3d',
 'blockHeight': 3714342,
 'blockHash': '0x13836b0e876d9fba0a3454605f0efa9ab9bc451c7046ac71dfc83d2ecf0ca32c',
 'txIndex': 1,
 'to': 'cx84858b30b7df25f70063c27c717243fa6c3d7580',
 'stepUsed': 147080,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 147080,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
params={
    "_account": NEWTON_ADDRESS,
    "_amount": 6,
}

call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("mintTo")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash


In [ ]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result


In [ ]:
params={
    "account": NEWTON_ADDRESS,
    "value": 5,
}

call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("_burn")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash


In [ ]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

In [ ]:
params={
    "owner": NEWTON_ADDRESS,
    "spender": prajwol.get_address(),
}

call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("_allowance")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash


In [ ]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

In [ ]:
params={
    "spender": NEWTON_ADDRESS,
    "amount": 3000,
}

call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("approve")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash


In [ ]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result